In [ ]:
import ast
import io
import json
import tokenize
from typing import Tuple, List, Any
import re

def extract_and_parse_prompt_list(code: str) -> List[Tuple[str, ...]]:
    """
    From a string of Python code, finds the first occurrence of
        = [ ... ]
    and parses that bracketed literal into a List[Tuple[str, ...]].

    Raises:
        ValueError if no list literal is found or it’s malformed.
    """
    # 1) grab everything from the first '=' up to the matching ']'
    m = re.search(r'=\s*(\[\s*[\s\S]*?\])', code)
    if not m:
        raise ValueError("No list literal found after an '=' in the code")
    list_str = m.group(1)

    # 2) safely evaluate it (only literals)
    try:
        data: Any = ast.literal_eval(list_str)
    except (SyntaxError, ValueError) as e:
        raise ValueError(f"Malformed list literal: {e}")

    # 3) validate shape
    if not isinstance(data, list) or not all(
        isinstance(item, (list, tuple)) and all(isinstance(x, str) for x in item) for item in data
    ):
        raise ValueError(
            "Parsed object is not a list of tuples/lists of strings"
        )

    # 4) convert to List[Tuple[str, ...]]
    return [tuple(str(x) for x in item) for item in data]


def _fix_quote_issues(code: str) -> str:
    """
    Fix common quote issues in LLM-generated code where
    inner quotes break string literals.
    - Converts single-quoted strings with inner single quotes to double-quoted.
    - Escapes quotes if needed.
    """
    def fix_string_literal(match: re.Match) -> str:
        s = match.group(0)  # full quoted string including quotes
        quote = s[0]        # ' or "
        inner = s[1:-1]

        if quote == "'":
            if "'" in inner and not re.search(r"\\'", inner):
                # switch outer to double quotes if safe
                if '"' not in inner:
                    return '"' + inner + '"'
                else:
                    # fall back to escaping
                    return "'" + inner.replace("'", "\\'") + "'"
        elif quote == '"':
            if '"' in inner and not re.search(r'\\"', inner):
                if "'" not in inner:
                    return "'" + inner + "'"
                else:
                    return '"' + inner.replace('"', '\\"') + '"'
        return s  # unchanged if no problem

    # Regex matches Python-like string literals (single-line only)
    string_pattern = r"(?:'[^'\\]*(?:\\.[^'\\]*)*'|\"[^\"\\]*(?:\\.[^\"\\]*)*\")"
    return re.sub(string_pattern, fix_string_literal, code)



forced_double_str = _fix_quote_issues("""
prompts = [
    (
        'Basophil cytoplasm is sparsely stained, appearing almost translucent due to the dense packing of granules.',
        'Eosinophil granules exhibit a distinct, uniform orange-red hue, often filling the majority of the cytoplasm.',
        'Lymphocyte cytoplasm is typically scant and intensely basophilic, appearing as a thin rim around the nucleus.',
        'Monocyte cytoplasm displays a characteristic 'ground glass' appearance with fine azurophilic granules and occasional pseudo-podia.',
        'Neutrophil cytoplasm contains fine, lilac-colored granules that are evenly distributed, but may show toxic granulation in infection.'
    ),
    (
        'Basophil nuclei are often obscured by the numerous, intensely stained granules, appearing as a dark, irregular mass.',
        'Eosinophil nuclei are typically bilobed, with each lobe connected by a prominent chromatin bridge.',
        'Lymphocyte nuclei are round or slightly indented, with densely packed chromatin and a visible nucleolus in activated cells.',
        'Monocyte nuclei are large and kidney-shaped, with a loose, vesicular chromatin pattern and a prominent nucleolus.',
        'Neutrophil nuclei exhibit a segmented pattern with 3-5 lobes, the number increasing with cell maturity.'
    ),
    (
        'Basophil cell size is consistently the smallest among granulocytes, typically measuring 7-9 μm in diameter.',
        'Eosinophil cell size is intermediate, ranging from 12-15 μm, and is often larger in individuals with parasitic infections.',
        'Lymphocyte cell size varies significantly with activation, ranging from 7-12 μm in resting cells to 15 μm or more in blast forms.',
        'Monocyte cell size is the largest of the leukocytes, typically measuring 14-20 μm, reflecting their phagocytic capacity.',
        'Neutrophil cell size is approximately 10-12 μm, with a consistent round to oval shape.'
    ),
    (
        'Basophil granules are metachromatic, exhibiting a deep purple-black color when stained with Wright-Giemsa.',
        'Eosinophil granules contain major basic protein, which is released during allergic reactions and parasitic infections.',
        'Lymphocyte granules, when present, are typically small and pale, representing stored cytokines or cytotoxic molecules.',
        'Monocyte cytoplasm contains numerous lysosomes, appearing as small, refractile granules responsible for intracellular digestion.',
        'Neutrophil granules contain enzymes like myeloperoxidase, contributing to their bactericidal activity.'
    ),
    (
        'Basophil granules are irregularly shaped and often coalesce, forming large clumps within the cytoplasm.',
        'Eosinophil granules are uniformly sized and spherical, creating a homogenous distribution throughout the cytoplasm.',
        'Lymphocyte cytoplasm is typically devoid of prominent granules, except in granular lymphocytes like NK cells.',
        'Monocyte cytoplasm is abundant and contains vacuoles of varying sizes, reflecting ongoing phagocytosis.',
        'Neutrophil granules are evenly distributed and appear as fine, lilac-colored specks throughout the cytoplasm.'
    ),
    (
        'Basophil nuclear chromatin is often condensed and obscured by the dense granule population.',
        'Eosinophil nuclear chromatin is typically evenly distributed and moderately condensed.',
        'Lymphocyte nuclear chromatin is densely packed, giving the nucleus a dark, intensely stained appearance.',
        'Monocyte nuclear chromatin is loose and vesicular, with a prominent nucleolus.',
        'Neutrophil nuclear chromatin is fine and delicate, connecting the segmented lobes of the nucleus.'
    ),
    (
        'Basophil cell shape is often distorted by the pressure of numerous granules, appearing irregular or lobulated.',
        'Eosinophil cell shape is generally round to oval, maintaining a consistent morphology.',
        'Lymphocyte cell shape is typically round, but can vary depending on activation state.',
        'Monocyte cell shape is often irregular, with pseudopodia extending from the cell surface.',
        'Neutrophil cell shape is round to slightly elongated, with a consistent morphology.'
    ),
    (
        'Basophil granules are responsible for releasing histamine and heparin, mediating inflammatory responses.',
        'Eosinophil granules contain proteins toxic to parasites, contributing to their role in parasitic infections.',
        'Lymphocyte granules contain perforin and granzymes, facilitating cytotoxic activity against infected cells.',
        'Monocyte granules contain lysosomal enzymes, enabling the breakdown of ingested pathogens and cellular debris.',
        'Neutrophil granules contain antimicrobial proteins, contributing to their role in bacterial killing.'
    )
]
""")
print(forced_double_str)


prompts = [
    (
        'Basophil cytoplasm is sparsely stained, appearing almost translucent due to the dense packing of granules.',
        'Eosinophil granules exhibit a distinct, uniform orange-red hue, often filling the majority of the cytoplasm.',
        'Lymphocyte cytoplasm is typically scant and intensely basophilic, appearing as a thin rim around the nucleus.',
        'Monocyte cytoplasm displays a characteristic 'ground glass' appearance with fine azurophilic granules and occasional pseudo-podia.',
        'Neutrophil cytoplasm contains fine, lilac-colored granules that are evenly distributed, but may show toxic granulation in infection.'
    ),
    (
        'Basophil nuclei are often obscured by the numerous, intensely stained granules, appearing as a dark, irregular mass.',
        'Eosinophil nuclei are typically bilobed, with each lobe connected by a prominent chromatin bridge.',
        'Lymphocyte nuclei are round or slightly indented, with densely packed chromat